In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pip install --upgrade gspread transformers
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==2.4.0
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 8.5 MB/s eta 0:00:00
  Preparing metada

In [ ]:
import gspread
from google.auth import default
import re
from transformers import BertJapaneseTokenizer
import time

# Google Sheets APIのクライアントを設定
creds, _ = default()
gc = gspread.authorize(creds)



# GoogleスプレッドシートのIDを入力（URLの "/d/" から "/edit" の間の部分）
spreadsheet_id = '1hOEd-KqETbR42Mj5AFGjsao6d2mN4dmgKeTkp1m1XmQ'


In [ ]:
# トークナイザーの設定
MODEL_NAME = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 正規表現パターン：評価部分を抽出するため
rating_pattern = r"5つ星のうち\d\.\d\s*"

# データの処理
def process_data(id,sheet_name):
  workbook = gc.open_by_key(id)
  worksheet = workbook.worksheet(sheet_name)
  data = worksheet.get_all_values()[1:]
  processed_data = []
  row_count = 1
  delete_row_list = []
  for row in data:
      title, review, category = row[0], row[1], row[2]

      # 1. タイトルの評価部分を削除
      title_cleaned = re.sub(rating_pattern, '', title).strip() + '\n'

      # タイトルの文字数に基づいてレビューの冒頭部分を取得
      title_length = len(title_cleaned)
      review_start = review[:title_length]
      after_title = f"{title_cleaned}"
      review_start2 = review[:len(after_title)]

      # レビューの冒頭部分にタイトルが含まれていない、またはレビューがタイトルのみの場合
      if title_cleaned not in review_start:
          review = f"{title_cleaned}\n{review}"

      # 2. B列のトークン数を確認し、512トークン以上の行はスキップ
      encoding = tokenizer(
            review,
            padding='do_not_pad',
            truncation=True,
            max_length=512
      )
      token_length = len(encoding['input_ids'])

      if token_length < 512:
        # トークン数が512を超える場合には512に収まるようにカット
        processed_data.append([title, review, category])
        #print(f"ID : {row_count}\n token_length : {token_length}")
      else:
        print(f"スキップされた行ID : {row_count}")
        delete_row_list.append(row_count)

      row_count += 1

  # 処理後のデータをスプレッドシートに上書き
  worksheet.update(range_name='A2', values=processed_data)  # 'A2'から書き込む
  # 行番号のリストを降順にソート
  delete_row_list.sort(reverse=True)

  # 各行を削除
  for row in delete_row_list:
      worksheet.delete_rows(row)
  time.sleep(2)  # 次のシート処理前に遅延を追加
  print(f"{sheet_name} : {len(delete_row_list)}件削除")

In [ ]:
for i in range(8):
  sheet_name = f"シート{i+1}"
  process_data(spreadsheet_id, sheet_name)

ID : 1
 token_length : 185
ID : 2
 token_length : 35
ID : 3
 token_length : 22
ID : 4
 token_length : 27
ID : 5
 token_length : 69
ID : 6
 token_length : 136
ID : 7
 token_length : 45
ID : 8
 token_length : 23
ID : 9
 token_length : 4
ID : 10
 token_length : 43
ID : 11
 token_length : 24
ID : 12
 token_length : 70
ID : 13
 token_length : 22
ID : 14
 token_length : 19
ID : 15
 token_length : 55
ID : 16
 token_length : 121
ID : 17
 token_length : 21
ID : 18
 token_length : 23
ID : 19
 token_length : 203
ID : 20
 token_length : 26
ID : 21
 token_length : 24
ID : 22
 token_length : 33
ID : 23
 token_length : 25
ID : 24
 token_length : 62
ID : 25
 token_length : 105
ID : 26
 token_length : 63
ID : 27
 token_length : 48
ID : 28
 token_length : 48
ID : 29
 token_length : 26
ID : 30
 token_length : 27
ID : 31
 token_length : 28
ID : 32
 token_length : 87
ID : 33
 token_length : 19
ID : 34
 token_length : 40
ID : 35
 token_length : 57
ID : 36
 token_length : 30
ID : 37
 token_length : 6
ID : 38

In [ ]:
#process_data(spreadsheet_id, "シート7")

スキップされた行：['5つ星のうち5.0 ■日本映画の最高傑作【原点にして頂点】', '最高\n\nこれに尽きます\n\n私は映画館で計３回鑑賞しました\n通常版⇒２回（万人向け）\nモノクロ版⇒１回（玄人向け）\n\nこの映画ジャンルを例えるなら､\nSF（サイエンス・フィクション）\nアクション､ドラマ､ミリタリー､ホラー\n\nゴジラ マイナスワンの劇場公開は、2023年11月3日で、文化の日（祝日）\nつまりは、初代ゴジラ（西暦1954年）の公開日と同じで、\n日本国で何事にもに文化され始めた記念すべき日に、\n公開日を合わせてきたことにもリスペクトを感じます\n\n個人的感想になりますが､\n物語としてかなり面白かったです\n特に新鮮さがありました\n\nこの映画を鑑賞するにあたり、\nひとつハードルがあるとすれば、\n人生経験と感受性（想像力）があるかを問われます\n（一部のお子様には不向きかもしれません）\n\n●この映画は先入観なしで観たほうが楽しめます\n私は､ほぼ予備知識を入れずに鑑賞しました\n（映画を観る礼儀みたいなもの）\n\nある程度の予備知識があると､より楽しめる事もありますが､全く知識が無くとも十分に楽しめます\n\nこの映画を一言で語るなら､\n『生きてゆく』ための模範\nそれを丁寧に描いた作品\nマニア向けを装いながら､\n実は万人受け（大衆受け）する作品\n\n見る者に感情を揺さぶる映画\nこれ一点に集約されます\n\n●怪獣の定義\n人（ココロからハミ出した感情）が生み出した産物（擬人化）\n（だから見た目で､どこか人型である必要がある）\n当初（初代ゴジラ）､着ぐるみだった理由はこれです\nこの基本を具現化したのがゴジラ（呉爾羅）\n\nーーーーーーーーーーーーーー\n戦中（WW2）⇒終戦直後を描いた物語\n戦争で生き残った人々が､各々その戦争の産物（ゴジラ）で負った傷痕を抱えながら､この世界でどう生き抜いてゆくのか？\nその背景を抱えながらキャスト陣によるドラマの連続で､目を外らす暇が無いため､必然的に釘付けになります\n\n本作は､東宝ゴジラ７０周年の作品になりますが､\n日本映画作品として恥じない､むしろ､国民として世界に誇るべきアイデンティティーの塊を感じる作品であったように思えます\n※東宝ゴジラの記念碑てき作品\